In [72]:
import requests


In [73]:
URL = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c"

list_of_lost_object = requests.get(URL)
data_collected = list_of_lost_object.json()

In [76]:
list_annee=[2016,2017,2018,2019,2020,2021]
list_data_collected_final = []

for i in list_annee:
    filtre = "&refine.date="+str(i)#pour selectionner seulement l'année
    for j in range(1,13):
        list_of_lost_object_annee = requests.get(URL+filtre+"%2F"+str(j)+"&rows=10000") #pour sélectionner seulement le mois ,1 pour janvier et 12 pour decembre
        list_data_collected_final.append(list_of_lost_object_annee.json())

# #On fait de cette manière car on ne peut que récuperer 10000 données à la fois

In [77]:
len(list_data_collected_final)

72

In [79]:
#Creation de la base de donnée
from sqlalchemy import create_engine, MetaData, \
    Column, Integer, Numeric, String, Date, Table, ForeignKey, select

engine = create_engine(
    'sqlite:///objet_trouve.sqlite3'
)
engine


Engine(sqlite:///objet_trouve.sqlite3)

In [80]:
metadata = MetaData()

In [81]:
import sqlalchemy
sqlalchemy.__version__ 

'1.4.45'

In [82]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
Base

sqlalchemy.orm.decl_api.Base

In [83]:
class ObjetTrouve(Base):
     __tablename__ = "Objet_trouve"
     record_id = Column(Integer, primary_key=True)
     id_gare_origine = Column(String)
     nom_gare_origine = Column(String)     
     type_objet_trouve = Column(String) 
     nature_objet_trouve = Column(String)
     date = Column(String)

     def __repr__(self):
         return "<Objet_trouve(record_id='%s', id_gare_origine='%s',nom_gare_origine='%s', type_objet_trouve='%s',nature_objet_trouve='%s',  date='%s')>" % (
             self.record_id,
             self.id_gare_origine,
             self.nom_gare_origine,
             self.type_objet_trouve,
             self.nature_objet_trouve,
             self.date,
         )
    

In [85]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
Session
Base.metadata.create_all(engine)

In [100]:
def generateObjet():
    """
    Fonction qui stocke les données de la SNCF dans la BDD
    """
    Session = sessionmaker(bind=engine)
    session = Session()
    for j in list_data_collected_final:
        for i in j["records"]:
            try:
                id_gare_origine_test =i['fields']['gc_obo_gare_origine_r_code_uic_c']
            except KeyError:
                id_gare_origine_test = "Pas d'Id de gare"
            try:
                nom_gare_origine_test =i['fields']['gc_obo_gare_origine_r_name']
            except KeyError:
                nom_gare_origine_test = "Pas d'Id de gare"
        

            objet_trouve_remplir = ObjetTrouve(id_gare_origine=id_gare_origine_test ,
            nom_gare_origine=nom_gare_origine_test, 
            type_objet_trouve=i['fields']['gc_obo_type_c'],
            nature_objet_trouve=i['fields']['gc_obo_nature_c'],
            date=i['fields']['date'])
            session.add(objet_trouve_remplir)
            
    session.commit()
    session.close()

In [101]:
generateObjet()